In [1]:
import tensorflow as tf

2025-01-10 17:28:35.361622: W tensorflow/stream_executor/platform/default/dso_loader.cc:60] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2025-01-10 17:28:35.361661: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
feature_columns = [
    "user_id",
    "prod_buy_cnt",
    "prod_pay_amt",
    "prod_permission_days",
    "adm_click_cnt",
    "refuse_call_num_7d",
    "k002602",
    "k001565",
    "k000431",
    "k5142",
    "k5137",
    "k1380",
    "k001336",
]
label_columns = [
    "is_called",
    "is_addv",
    "is_paid",
]
# 定义列名（根据 CSV 文件的列）
column_names = feature_columns + label_columns

## CSV加载方式
1.  单文件：`file_pattern = "/read-only/sample/folder_6661/sample_6661_1/20241014.txt"`
2.  文件列表：`file_pattern = "/read-only/sample/folder_6661/sample_6661_1/*.txt"`
3.  通配符模式：`file_pattern = ["data1.csv", "data2.csv"]`

In [3]:
# 使用方式没有区别
file_pattern = "/read-only/sample/folder_6661/sample_6661_1/20241014.txt"

In [4]:
# 定义列的数据类型：手动指定
column_defaults = [tf.string] * len(feature_columns) + [tf.int32] * len(
    label_columns
)

In [5]:
batch_size = 32

## 数据集构建

- `make_csv_dataset`仅支持单列作为label
- 多列需要特殊处理


In [6]:
dataset = tf.data.experimental.make_csv_dataset(
    file_pattern,
    batch_size=batch_size,  # 每次读取的批次大小
    column_names=column_names,  # 列名
    field_delim="\t",
    column_defaults=column_defaults,  # 列的数据类型 (不加会自动推断，容易出错)
    label_name="is_called",  # 标签列名( 仅支持单个标签)
    num_epochs=1,  # 数据重复次数（None 表示无限重复）
    shuffle=True,  # 是否打乱数据
    shuffle_buffer_size=1000,  # 打乱缓冲区大小
    ignore_errors=True,  # 忽略解析错误
)

for batch in dataset.take(1):
    feat = batch[0]
    label = batch[1]
    for k, v in feat.items():
        print("Features:", k, v.shape, v.dtype)

    print("Labels:", label.shape, label.dtype)

Features: user_id (32,) <dtype: 'string'>
Features: prod_buy_cnt (32,) <dtype: 'string'>
Features: prod_pay_amt (32,) <dtype: 'string'>
Features: prod_permission_days (32,) <dtype: 'string'>
Features: adm_click_cnt (32,) <dtype: 'string'>
Features: refuse_call_num_7d (32,) <dtype: 'string'>
Features: k002602 (32,) <dtype: 'string'>
Features: k001565 (32,) <dtype: 'string'>
Features: k000431 (32,) <dtype: 'string'>
Features: k5142 (32,) <dtype: 'string'>
Features: k5137 (32,) <dtype: 'string'>
Features: k1380 (32,) <dtype: 'string'>
Features: k001336 (32,) <dtype: 'string'>
Features: is_addv (32,) <dtype: 'int32'>
Features: is_paid (32,) <dtype: 'int32'>
Labels: (32,) <dtype: 'int32'>


2025-01-10 17:28:39.141471: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2025-01-10 17:28:39.142753: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2025-01-10 17:28:39.183997: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:02:00.0 name: NVIDIA GeForce GTX 1080 computeCapability: 6.1
coreClock: 1.835GHz coreCount: 20 deviceMemorySize: 7.93GiB deviceMemoryBandwidth: 298.32GiB/s
2025-01-10 17:28:39.184820: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 1 with properties: 
pciBusID: 0000:82:00.0 name: NVIDIA GeForce GTX 1080 computeCapability: 6.1
coreClock: 1.835GHz coreCount: 20 deviceMemorySize: 7.93GiB deviceMemoryBandwidth: 298.32GiB/s
2025-01-10 17:28:39.185235: W tensorflow/stream_executor/platform/default/dso_loader.cc:60] Could not load dynamic library 'libcudart.so.11.0'; d

## 删除某一列

In [7]:
from functools import partial

dataset = tf.data.experimental.make_csv_dataset(
    file_pattern,
    batch_size=batch_size,  # 每次读取的批次大小
    column_names=column_names,  # 列名
    field_delim="\t",
    column_defaults=column_defaults,  # 列的数据类型 (不加会自动推断，容易出错)
    label_name="is_called",  # 标签列名( 仅支持单个标签)
    num_epochs=1,  # 数据重复次数（None 表示无限重复）
    shuffle=True,  # 是否打乱数据
    shuffle_buffer_size=1000,  # 打乱缓冲区大小
    ignore_errors=True,  # 忽略解析错误
)

def drop_columns(features:dict, labels, drop_columns):
    for col in drop_columns:
        features.pop(col, None )
    return features, labels
drop_columns_with_name = partial(
    drop_columns, drop_columns=['user_id']
)
dataset = dataset.map(lambda x, y: drop_columns_with_name(x, y))


for batch in dataset.take(1):
    feat = batch[0]
    label = batch[1]
    for k, v in feat.items():
        print("Features:", k, v.shape, v.dtype)

    print("Labels:", label.shape, label.dtype)

Features: prod_buy_cnt (32,) <dtype: 'string'>
Features: prod_pay_amt (32,) <dtype: 'string'>
Features: prod_permission_days (32,) <dtype: 'string'>
Features: adm_click_cnt (32,) <dtype: 'string'>
Features: refuse_call_num_7d (32,) <dtype: 'string'>
Features: k002602 (32,) <dtype: 'string'>
Features: k001565 (32,) <dtype: 'string'>
Features: k000431 (32,) <dtype: 'string'>
Features: k5142 (32,) <dtype: 'string'>
Features: k5137 (32,) <dtype: 'string'>
Features: k1380 (32,) <dtype: 'string'>
Features: k001336 (32,) <dtype: 'string'>
Features: is_addv (32,) <dtype: 'int32'>
Features: is_paid (32,) <dtype: 'int32'>
Labels: (32,) <dtype: 'int32'>


## 多列作为label


In [8]:
from functools import partial

dataset = tf.data.experimental.make_csv_dataset(
    file_pattern,
    batch_size=batch_size,  # 每次读取的批次大小
    column_names=column_names,  # 列名
    field_delim="\t",
    column_defaults=column_defaults,  # 列的数据类型 (不加会自动推断，容易出错)
    # label_name="is_called",  # 标签列名( 仅支持单个标签)
    num_epochs=1,  # 数据重复次数（None 表示无限重复）
    shuffle=True,  # 是否打乱数据
    shuffle_buffer_size=1000,  # 打乱缓冲区大小
    ignore_errors=True,  # 忽略解析错误
)    
def extract_multi_labels(features, label_columns):
        feat = {k: v for k, v in features.items() if k not in label_columns}
        label = {k: v for k, v in features.items() if k in label_columns}
        return feat, label


extract_multi_labels_with_name = partial(
    extract_multi_labels, label_columns=label_columns
)
dataset = dataset.map(lambda x: extract_multi_labels_with_name(x))


for batch in dataset.take(1):
    feat = batch[0]
    label = batch[1]
    for k, v in feat.items():
        print("Features:", k, v.shape, v.dtype)
    for k, v in label.items():
        print("Labels:", k, v.shape, v.dtype)

Features: user_id (32,) <dtype: 'string'>
Features: prod_buy_cnt (32,) <dtype: 'string'>
Features: prod_pay_amt (32,) <dtype: 'string'>
Features: prod_permission_days (32,) <dtype: 'string'>
Features: adm_click_cnt (32,) <dtype: 'string'>
Features: refuse_call_num_7d (32,) <dtype: 'string'>
Features: k002602 (32,) <dtype: 'string'>
Features: k001565 (32,) <dtype: 'string'>
Features: k000431 (32,) <dtype: 'string'>
Features: k5142 (32,) <dtype: 'string'>
Features: k5137 (32,) <dtype: 'string'>
Features: k1380 (32,) <dtype: 'string'>
Features: k001336 (32,) <dtype: 'string'>
Labels: is_called (32,) <dtype: 'int32'>
Labels: is_addv (32,) <dtype: 'int32'>
Labels: is_paid (32,) <dtype: 'int32'>


## 数据集拆分


In [9]:
# 定义验证集大小
validation_size = 10  # 直接指定验证集大小：10 x batch_size 行数据

# 拆分数据集
validation_dataset = dataset.take(
    validation_size
)  # 取前 validation_size 条数据作为验证集
train_dataset = dataset.skip(
    validation_size
)  # 跳过前 validation_size 条数据作为训练集

# 打印数据集大小
print(f"Train dataset size: {sum(1 for _ in train_dataset) * batch_size}")
print(f"Validation dataset size: {sum(1 for _ in validation_dataset) * batch_size}")

Train dataset size: 48288
Validation dataset size: 320
